# 4.4.4 Polynomial Regression
We can now explore these concepts interactively by fitting polynomials to data.

In [1]:
##import math
##from mxnet import gluon, np, npx
##from mxnet.gluon import nn
##from d2l import mxnet as d2l
##npx.set_np()

use strict;
use warnings;
use Data::Dump qw(dump);
use AI::MXNet qw(mx);
use AI::MXNet::Gluon qw(gluon);
use List::Util qw(min max shuffle);
use d2l;
use d2l::Accumulator;
use d2l::Animator;

### Generating the Dataset
First we need data. Given x, we will use the following cubic polynomial to generate the labels on
training and test data:

$$ y = 5 + 1.2x − 3.4\frac{x^2}{2!} + 5.6\frac{x^3}{3!} + ϵ $$
$$ where: ϵ ∼ \mathcal{N} (0, {0.1}^{2}) $$
The noise term ϵ obeys a normal distribution with a mean of 0 and a standard deviation of 0.1. For
optimization, we typically want to avoid very large values of gradients or losses. This is why the
features are rescaled from xi to xii!. It allows us to avoid very large values for large exponents i. We
will synthesize 100 samples each for the training set and test set.

In [2]:
sub gamma {
    ##source code: (https://hewgill.com/picomath/perl/gamma.pl.html)
    my $x = $_[0];

    if ($x <= 0.0){
        die "Invalid input argument $x. Argument must be positive";
    }

    my $gamma = 0.577215664901532860606512090; # Euler's gamma constant

    if ($x < 0.001) {
        return 1.0/($x*(1.0 + $gamma*$x));
    }

    if ($x < 12.0)
    {
        my $y = $x;
        my $n = 0;
        my $arg_was_less_than_one = ($y < 1.0);

        if ($arg_was_less_than_one){
            $y += 1.0;
        }else{
            $n = int($y) - 1;  # will use n later
            $y -= $n;
        }

        my @p =
        (
            -1.71618513886549492533811E+0,
             2.47656508055759199108314E+1,
            -3.79804256470945635097577E+2,
             6.29331155312818442661052E+2,
             8.66966202790413211295064E+2,
            -3.14512729688483675254357E+4,
            -3.61444134186911729807069E+4,
             6.64561438202405440627855E+4
        );
        my @q =
        (
            -3.08402300119738975254353E+1,
             3.15350626979604161529144E+2,
            -1.01515636749021914166146E+3,
            -3.10777167157231109440444E+3,
             2.25381184209801510330112E+4,
             4.75584627752788110767815E+3,
            -1.34659959864969306392456E+5,
            -1.15132259675553483497211E+5
        );

        my $num = 0.0;
        my $den = 1.0;
        my $i;

        my $z = $y - 1;
        for ($i = 0; $i < 8; $i++){
            $num = ($num + $p[$i])*$z;
            $den = $den*$z + $q[$i];
        }
        my $result = $num/$den + 1.0;

        if ($arg_was_less_than_one){
            $result /= ($y-1.0);
        }else{
            for ($i = 0; $i < $n; $i++) {
                $result *= $y++;
            }
        }
        return $result;
    }

    if ($x > 171.624){
        return undef;
    }

    return exp(log_gamma($x));
}

sub log_gamma {
    my $x = $_[0];

    if ($x <= 0.0)
    {
        die "Invalid input argument $x. Argument must be positive";
    }

    if ($x < 12.0)
    {
        return log(abs(gamma($x)));
    }

    # Abramowitz and Stegun 6.1.41
    # Asymptotic series should be good to at least 11 or 12 figures
    # For error analysis, see Whittiker and Watson
    # A Course in Modern Analysis (1927), page 252

    my @c =
    (
         1.0/12.0,
        -1.0/360.0,
         1.0/1260.0,
        -1.0/1680.0,
         1.0/1188.0,
        -691.0/360360.0,
         1.0/156.0,
        -3617.0/122400.0
    );
    my $z = 1.0/($x*$x);
    my $sum = $c[7];
    for (my $i=6; $i >= 0; $i--)
    {
        $sum *= $z;
        $sum += $c[$i];
    }
    my $series = $sum/$x;

    my $halfLogTwoPi = 0.91893853320467274178032973640562;
    my $logGamma = ($x - 0.5)*log($x) - $x + $halfLogTwoPi + $series;    
    return $logGamma;
}

In [3]:
sub getSubND{
    my ($input) = @_;
       if(!$input->{data}){
        print "ERR: No ha colocado el array";
        return 1;
    }
    if(!$input->{row_end}){
        print "ERR: No ha colocado el valor de row_end";
        return 1;
    }else{
        if(!$input->{row_start}){
            $input->{row_start} = 0;
        }
    }
    
    if($input->{column_end} && $input->{data}->shape->[1]){
        if(!$input->{column_start}){
            $input->{column_start} = 0;
        }    
        my @arr;
        for my $i ($input->{row_start}..$input->{row_end}-1){
            push(@arr, $input->{data}->[$i]->_slice($input->{column_start}, $input->{column_end})->asarray);
        }
        return mx->nd->array(\@arr);
    }else{
        if(!$input->{data}->shape->[1]){
            return $input->{data}->_slice($input->{row_start}, $input->{row_end});
        }else{
            print "ERR: No ha colocado el valor de column_end";
            return 1;        
        }
    }
}

In [4]:
my $max_degree = 20; ## Maximum degree of the polynomial
my ($n_train, $n_test) = (100, 100); ## Training and test dataset sizes
my $true_w = mx->nd->zeros([$max_degree]); ## Allocate lots of empty space
$true_w = $true_w->asarray;
@$true_w[0..3] = (5, 1.2, -3.4, 5.6);
$true_w = mx->nd->array($true_w);

my $features = mx->nd->random->normal(shape=>[$n_train + $n_test, 1]);
$features = mx->nd->shuffle($features);

my $poly_features = $features ** mx->nd->arange(stop=>$max_degree)->reshape([1,-1]);

my $len = $poly_features->shape->[0];
$poly_features = $poly_features->asarray;
for my $i (0..$max_degree-1){
    my $gamma = gamma($i + 1);
    for my $j (0..$len-1){
        $$poly_features[$j][$i] = $$poly_features[$j][$i] / $gamma;
    }
}
$poly_features = mx->nd->array($poly_features);

my $labels = mx->nd->dot($poly_features, $true_w);
$labels += mx->nd->random->normal(scale=>0.1, shape=>$labels->shape);

<AI::MXNet::NDArray 200 @cpu(0)>

Again, monomials stored in poly_features are rescaled by the gamma function, where Γ(n) =
(n − 1)!. Take a look at the first 2 samples from the generated dataset. The value 1 is technically a
feature, namely the constant feature corresponding to the bias.

In [5]:
##print dump features[:2], poly_features[:2, :], labels[:2]
print dump(getSubND({data=>$features, row_start=>0, row_end=>2, column_end=>1})->asarray);
print "\n";
print dump((getSubND({data=>$poly_features, row_start=>0, column_start=>0, row_end=>2, column_end=>$poly_features->shape->[1]}))->asarray);
print "\n";
print dump((getSubND({data=>$labels, row_start=>0, row_end=>2}))->asarray);
print "\n";

[[1.50947511196136], [1.96766126155853]]
[
  [
    1,
    1.50947511196136,
    1.13925755023956,
    0.573226988315582,
    0.216317966580391,
    0.0653053149580956,
    0.0164294578135014,
    0.00354283698834479,
    0.000668478023726493,
    0.000112116766104009,
    1.6923748262343e-05,
    2.32236129704688e-06,
    2.921289024016e-07,
    3.39200987298227e-08,
    3.65725338902223e-09,
    3.68035518727439e-10,
    3.47212814055808e-11,
    3.08299458011418e-12,
    2.58539066891328e-13,
    2.05399120021404e-14,
  ],
  [
    1,
    1.96766126155853,
    1.93584537506104,
    1.26969599723816,
    0.624582946300507,
    0.245793521404266,
    0.0806063935160637,
    0.0226580128073692,
    0.00557291181758046,
    0.00121840031351894,
    0.000239739893004298,
    4.28842649853323e-05,
    7.03180876371334e-06,
    1.06432446500548e-06,
    1.49587862097178e-07,
    1.96225489190738e-08,
    2.41315811777554e-09,
    2.79310463646709e-10,
    3.05326874894263e-11,
    3.16199909

1

### Training and Testing the Model
Let us first implement a function to evaluate the loss on a given dataset.

In [6]:
sub evaluate_loss{
    my ($net, $data_iter, $loss) = @_;
    #"""Evaluate the loss of a model on the given dataset."""
    my $metric = Accumulator->new(2); # Sum of losses, no. of examples
    my ($X, $y, $accuracy, $l);
    while (defined(my $batch = $data_iter->())){
        $X = $batch->{data};
        $y = $batch->{label}->astype('float32');
        
        $l = $loss->($net->($X), $y);
        $metric->add([ $l->sum(), $l->size]);
    }
    if($metric->getitem(1)==0){
        return (0);
    }else{
        return ($metric->getitem(0) / $metric->getitem(1));
    }
}

Now define the training function.

In [7]:
sub train_epoch_ch3{ #@save OPTIONAL - TESTING
    my ($net, $train_iter, $loss, $updater) = @_;
    my $metric = Accumulator->new(3);
    $updater = $updater->step(10,1);

    
    while (defined(my $batch = $train_iter->())){
        our $X = $batch->{data};
        our $y = $batch->{label}->astype('float32');
    
        our $y_hat;
        our $l;
        mx->autograd->record(sub {
            $y_hat = $net->($X);
            $l = $loss->($y_hat, $y);
        });
        $l->backward;
        $updater->step($X->shape->[0], 1);
        
        $metric->add([$l->sum()->astype('float32'), accuracy($y_hat, $y), $y->size])
    }
    
    if($metric->getitem(2)==0){
        return (0);
    }else{
        return ($metric->getitem(0) / $metric->getitem(2), $metric->getitem(1) / $metric->getitem(2));
    }
}

In [8]:
sub data_iter_sequential{ # Optimized for sequential minibatches
  my ($features, $labels, $batch_size) = @_;
  my $num_samples = $features->len;
  my @indices = (0 .. $num_samples - 1);
  my ($index, @batch_indices) = 0;

  return sub {
    if (defined $_[0] && $_[0] == 0){# Reset
      $index = 0;
      return 1;
    }
    return undef if ($index >= $num_samples);
    @batch_indices = @indices[$index .. min($index + $batch_size, $num_samples) - 1];
    $index += $batch_size;
    return {data  => $features->slice([$batch_indices[0], $batch_indices[-1]]), 
            label => $labels->slice([$batch_indices[0], $batch_indices[-1]])};
  };
}

In [9]:
sub train{
    my ($train_features, $test_features, $train_labels, $test_labels, $num_epochs) = @_;
    if(!$num_epochs) {$num_epochs = 2;}

    my $batch_size = min(10, $train_labels->shape->[0]);
    my $loss = gluon->loss->L2Loss();
    my $net = gluon->nn->Sequential();
    $net->name_scope(sub {
        $net->add(gluon->nn->Dense(1, use_bias=>0, in_units=>4))
        });
    # Switch off the bias since we already catered for it in the polynomial features
    $net->initialize();
    
    ##my $train_iter = d2l->load_array([$train_features, $train_labels], $batch_size);
    my $train_iter = data_iter_sequential($train_features, $train_labels, $batch_size);
    ##my $test_iter = d2l->load_array([$test_features, $test_labels], $batch_size);#, is_train=>0);
    my $test_iter = data_iter_sequential($test_features, $test_labels, $batch_size);
    ##print dump $train_iter->();
    my $trainer = gluon->Trainer($net->collect_params(), 'sgd', {learning_rate => 0.01});

    my $animator = Animator->new(xlabel=>'epoch', ylabel=>'loss', yscale=>'logarithm', xlim=>[1, $num_epochs], ylim=>[1e-3, 1e2], legend=>['train', 'test']);
    
    for my $epoch (0..$num_epochs-1){
        print $epoch . "\n";
        train_epoch_ch3($net, $train_iter, $loss, $trainer);        
        if( $epoch == 0 || ($epoch + 1) % 20 == 0){
            $animator->add($epoch + 1, (evaluate_loss($net, $train_iter, $loss), evaluate_loss($net, $test_iter, $loss)))
        }         
    }

    #print('weight:', $net->[0]->weight->data()->asnumpy())
}

### Third-Order Polynomial Function Fitting (Normal)
We will begin by first using a third-order polynomial function, which is the same order as that
of the data generation function. The results show that this modelʼs training and test losses can
be both effectively reduced. The learned model parameters are also close to the true values w =
[5, 1.2, −3.4, 5.6].

In [ ]:
# Pick the first four dimensions, i.e., 1, x, x^2/2!, x^3/3! from the
# polynomial features
train(
    getSubND({data=>$poly_features, row_start=>0, column_start=>0, row_end=>$n_train, column_end=>4}),
    getSubND({data=>$poly_features, row_start=>$n_train+1, column_start=>0, row_end=>$poly_features->shape->[0], column_end=>4}),
    getSubND({data=>$labels, row_start=>0, row_end=>$n_train}),
    getSubND({data=>$labels, row_start=>$n_train+1, row_end=>$labels->shape->[0]})
)

### Linear Function Fitting (Underfitting)
Let us take another look at linear function fitting. After the decline in early epochs, it becomes
difficult to further decrease this modelʼs training loss. After the last epoch iteration has been
completed, the training loss is still high. When used to fit nonlinear patterns (like the third-order
polynomial function here) linear models are liable to underfit.

In [ ]:
# Pick the first two dimensions, i.e., 1, x, from the polynomial features
train(
    getSubND({data=>$poly_features, row_start=>0, column_start=>0, row_end=>$n_train, column_end=>2}),
    getSubND({data=>$poly_features, row_start=>$n_train+1, column_start=>0, row_end=>$poly_features->shape->[0], column_end=>2}),
    getSubND({data=>$labels, row_start=>0, row_end=>$n_train}),
    getSubND({data=>$labels, row_start=>$n_train+1, row_end=>$labels->shape->[0]})
)


### Higher-Order Polynomial Function Fitting (Overfitting)
Now let us try to train the model using a polynomial of too high degree. Here, there are insufficient
data to learn that the higher-degree coefficients should have values close to zero. As a result, our
overly-complex model is so susceptible that it is being influenced by noise in the training data.
Though the training loss can be effectively reduced, the test loss is still much higher. It shows that
the complex model overfits the data.

In [ ]:
# Pick all the dimensions from the polynomial features
train(
    getSubND({data=>$poly_features, row_start=>0, column_start=>0, row_end=>$n_train, column_end=>$poly_features->shape->[1]}),
    getSubND({data=>$poly_features, row_start=>$n_train+1, column_start=>0, row_end=>$poly_features->shape->[0], column_end=>$poly_features->shape->[1]}),
    getSubND({data=>$labels, row_start=>0, row_end=>$n_train}),
    getSubND({data=>$labels, row_start=>$n_train+1, row_end=>$labels->shape->[0]}),
    1500
)

In the subsequent sections, we will continue to discuss overfitting problems and methods for dealing with them, such as weight decay and dropout.

### Summary
* Since the generalization error cannot be estimated based on the training error, simply minimizing the training error will not necessarily mean a reduction in the generalization error. <br> Machine learning models need to be careful to safeguard against overfitting so as to minimize the generalization error.
* A validation set can be used for model selection, provided that it is not used too liberally.
* Underfitting means that a model is not able to reduce the training error. When training error is much lower than validation error, there is overfitting.
* We should choose an appropriately complex model and avoid using insufficient training samples.